In [1]:
import pyaudio

# Connect Bluetooth

In [1]:
import socket


In [3]:

serverMACAddress = '98:D3:71:FD:A9:6F'  # Put your HC-05 address here
port = 1  # Match the setting on the HC-05 module
s = socket.socket(socket.AF_BLUETOOTH, socket.SOCK_STREAM, socket.BTPROTO_RFCOMM)
s.connect((serverMACAddress,port))


In [5]:
print("Connected. Type something...")


Connected. Type something...


In [6]:
def send_bl(text):
    s.send(bytes(text, 'UTF-8'))
    s.send(bytes(text, 'UTF-8'))
    

# Speech-to-text API

In [10]:
def give_com(MyText):
    if MyText=='right':
        send_bl('R2')
        send_bl('R2')

    elif MyText=='left' or MyText=='lyft':
        send_bl('L2')
        send_bl('L2')
    elif MyText=='go' or MyText=='google':
        send_bl('F2')
        send_bl('F2')
    elif MyText=='backward' or MyText=='back':
        send_bl('B2')
        send_bl('B2')
    elif MyText=='stop':
        send_bl('S')
        send_bl('S')

    else:
        send_bl('S')

In [11]:
# Python program to translate
# speech to text and text to speech


import speech_recognition as sr
import pyttsx3
import time

# Initialize the recognizer
r = sr.Recognizer()

# # Function to convert text to
# # speech
# def SpeakText(command):
	
# 	# Initialize the engine
# 	engine = pyttsx3.init()
# 	engine.say(command)
# 	engine.runAndWait()
	
	
# # Loop infinitely for user to
# speak
m=sr.Microphone()

In [9]:

while(1):

        # use the microphone as source for input.
        with m as source2:
            print('say smthing')
            # wait for a second to let the recognizer
            # adjust the energy threshold based on
            # the surrounding noise level
            t=time.time()
            r.adjust_for_ambient_noise(source2, duration=0.8)
            #listens for the user's input
#             audio2 = r.listen(source2,)
            audio2=r.record(source2, duration=4)
#             print(audio2)
            # Using google to recognize audio
            # try:
            MyText = r.recognize_google(audio2)
            # except:
            #     print('repeat')
            #     audio2=r.record(source2, duration=3)
            #     MyText = r.(audio2)
            


            MyText = MyText.lower()
            give_com(MyText)
            # MyText = r.recognize_google(audio2)
            # give_com(MyText.lower())


            time.sleep(1.5)
            print("Did you say "+MyText)
            print(time.time()-t)

In [ ]:
# s.close()

In [14]:
def give_single():
        with m as source2:
                print('say smthing')
                # wait for a second to let the recognizer
                # adjust the energy threshold based on
                # the surrounding noise level
                # t=time.time()
                r.adjust_for_ambient_noise(source2, duration=0.3)
                #listens for the user's input
        #             audio2 = r.listen(source2,)
                audio2=r.record(source2, duration=3)
        #             print(audio2)
                # Using google to recognize audio
                try:
                        MyText = r.recognize_google(audio2)
                except:
                    print('repeat')
                    audio2=r.record(source2, duration=3)
                    MyText = r.recognize_google(audio2)
                    

                MyText = MyText.lower()
                give_com(MyText)
        # MyText = r.recognize_google(audio2)
        # give_com(MyText.lower())


        # time.sleep(1.5)
        print("Did you say "+MyText)
        # print(time.time()-t)

In [20]:
# give_single()

say smthing
Did you say left


## TF Model

In [16]:
import tensorflow as tf

In [48]:
import sounddevice as sd
import numpy as np
import scipy.signal
import timeit
import python_speech_features


# Parameters
debug_time = 1
debug_acc = 0
led_pin = 8
word_threshold = 0.5
rec_duration = 0.5
window_stride = 2
sample_rate = 48000
resample_rate = 8000
num_channels = 1
num_mfcc = 16
model_path = 'D:\Graduating Senior\Spring\Embedded\Project\speech_model.h5'
model=tf.keras.models.load_model(model_path)

# Sliding window
window = np.zeros(int(rec_duration * resample_rate) * 2)

# GPIO 
# GPIO.setwarnings(False)
# GPIO.setmode(GPIO.BOARD)
# GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)

# # Load model (interpreter)
# interpreter = Interpreter(model_path)
# interpreter.allocate_tensors()
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()
# print(input_details)

# Decimate (filter and downsample)
def decimate(signal, old_fs, new_fs):
    
    # Check to make sure we're downsampling
    if new_fs > old_fs:
        print("Error: target sample rate higher than original")
        return signal, old_fs
    
    # We can only downsample by an integer factor
    dec_factor = old_fs / new_fs
    if not dec_factor.is_integer():
        print("Error: can only decimate by integer factor")
        return signal, old_fs

    # Do decimation
    resampled_signal = scipy.signal.decimate(signal, int(dec_factor))

    return resampled_signal, new_fs

def sd_callback(rec, frames, time, status):

#     GPIO.output(led_pin, GPIO.LOW)

    # Start timing for testing
    start = timeit.default_timer()
    
    # Notify if errors
    if status:
        print('Error:', status)
    
    # Remove 2nd dimension from recording sample
    rec = np.squeeze(rec)
    # print(rec)
    # Resample
    rec, new_fs = decimate(rec, sample_rate, resample_rate)
    
    # Save recording onto sliding window
    window[:len(window)//2] = window[len(window)//2:]
    window[len(window)//2:] = rec

    # Compute features
    mfccs = python_speech_features.base.mfcc(window, 
                                        samplerate=new_fs,
                                        winlen=0.256,
                                        winstep=0.050,
                                        numcep=num_mfcc,
                                        nfilt=26,
                                        nfft=2048,
                                        preemph=0.0,
                                        ceplifter=0,
                                        appendEnergy=False,
                                        winfunc=np.hanning)
    mfccs = mfccs.transpose()
#     print(mfccs)
    # for m in mfccs:
    #     plt.scatter(list(range(len(m))),m)
    #     plt.show()
    # Make prediction from model
    in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
    preds=model.predict(in_tensor,verbose=False)
    print(np.argmax(preds.ravel()))
# Start streaming from microphone
with sd.InputStream(channels=num_channels,
                    samplerate=sample_rate,
                    blocksize=int(sample_rate * rec_duration),
                    callback=sd_callback):
    while True:
        pass

0
0
0
0
0
0


KeyboardInterrupt: 

In [14]:
import matplotlib.pyplot as plt


In [26]:
from scipy.io.wavfile import read

In [49]:
f_path='left_1.wav'
sr,wv=read(f_path)

In [50]:
sr

16000

In [51]:
rec=wv
rec = np.squeeze(rec)
print(rec)
# Resample
rec, new_fs = decimate(rec, 16000, 4000)

# Save recording onto sliding window
window[:len(window)//2] = window[len(window)//2:]
window[len(window)//2:] = rec

# Compute features
mfccs = python_speech_features.base.mfcc(window, 
                                    samplerate=new_fs,
                                    winlen=0.256,
                                    winstep=0.050,
                                    numcep=num_mfcc,
                                    nfilt=26,
                                    nfft=2048,
                                    preemph=0.0,
                                    ceplifter=0,
                                    appendEnergy=False,
                                    winfunc=np.hanning)
mfccs = mfccs.transpose()
#     print(mfccs)
# for m in mfccs:
#     plt.scatter(list(range(len(m))),m)
#     plt.show()
# Make prediction from model
in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
preds=model.predict(in_tensor,verbose=False)
print(np.argmax(preds))

[  1   1   1 ... 330 324 321]
6


In [56]:
preds[2].argmax()

0